In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import models,transforms
import matplotlib.pyplot as plt
import pickle
from collections import OrderedDict
import csv
import collections
from  PIL import Image
from tqdm.notebook import tqdm_notebook
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')
import math
device = torch.device("mps" if torch.has_mps else "cpu")

In [4]:
test_df = pd.read_csv('data/FairFace_Paper/Labels/fairface_label_val.csv')
test_df

,file,age,gender,race,service_test
0,val/1.jpg,3-9,Male,East Asian,False
1,val/2.jpg,50-59,Female,East Asian,True
2,val/3.jpg,30-39,Male,White,True
3,val/4.jpg,20-29,Female,Latino_Hispanic,True
4,val/5.jpg,20-29,Male,Southeast Asian,False
...,...,...,...,...,...
10949,val/10950.jpg,30-39,Male,White,True
10950,val/10951.jpg,50-59,Male,White,False
10951,val/10952.jpg,60-69,Male,Latino_Hispanic,False
10952,val/10953.jpg,20-29,Female,East Asian,False


In [6]:
train_df = pd.read_csv('data/FairFace_Paper/Labels/fairface_label_train.csv')
train_df

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True
...,...,...,...,...,...
86739,train/86740.jpg,20-29,Male,Indian,True
86740,train/86741.jpg,10-19,Male,Indian,True
86741,train/86742.jpg,more than 70,Female,Indian,True
86742,train/86743.jpg,10-19,Female,Black,True


In [5]:
test_img_path = 'data/FairFace_Paper/Images/fairface-img-margin025-trainval'

0.20624562
0.0
3.6953487
3.2545228
1.1287235
0.0
0.0
0.0
0.0
0.09527007
0.0
0.0
9.173287
0.03562373
0.19500744
4.2282743
0.059036966
0.08198613
0.054575413
0.0
0.0
9.850888
0.87447375
0.25716493
0.13720155
1.5294275
2.1206112
0.0
2.0775857
0.0
0.0
0.62913597
0.0
2.727776
2.0635686
0.31828842
0.09418299
1.4818282
1.5778563
0.3678565
0.08241942
0.0
0.0
0.4888509
1.4769729
0.0
4.394194
0.0
0.501898
6.289736
0.0
0.02733304
0.0
0.06789268
0.0
0.26609015
0.02100636
0.9579458
0.0
0.3542837
0.0
0.0
3.8666816
8.945289
0.0057433047
2.9045856
0.0
1.817663
0.80926317
0.092777304
5.559615
0.2990454
1.1844766
0.0
0.10111741
0.037089597
2.5681949
0.42368647
0.12320322
0.8503663
0.6104985
1.6458669
0.0
0.55949634
0.0
8.288217
0.0
0.06908778
0.0
0.003446591
0.0
0.0
0.0
0.800624
0.015896954
0.0
0.0
0.96564335
3.826526
0.0
2.827526
0.0
2.2607653
0.0
5.969431
1.6490141
0.0
1.7015773
0.0
0.0019148294
0.099853665
0.0
0.5587835
1.8328991
0.60889965
9.81599
0.26633292
1.3357178
7.1996493
1.0317559
0.009160188